In [1]:
import pandas as pd
from datetime import date, timedelta
from sqlalchemy import create_engine

engine = create_engine('mysql+pymysql://root:@localhost:3306/portfolio_development')
conpf = engine.connect()

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"

format_dict = {'qty':'{:,}',
              'sell_price':'{:,.2f}','buy_price':'{:,.2f}','diff':'{:,.2f}',
              'sell_amt':'{:,.2f}','buy_amt':'{:,.2f}','gross':'{:,.2f}',
              'pct':'{:,.2f}','net':'{:,.2f}'}
pd.set_option('display.float_format','{:,.2f}'.format)
year = 2022
refund = 0.00
dividend = 0.00
expense  = 20_000.00

today = date.today()
today

datetime.date(2022, 1, 10)

### Restart and Run All Cells

### Record selection for transactions

In [2]:
sql = '''
SELECT name, YEAR(sells.date) AS sell_year, MONTH(sells.date) AS sell_month, 
       YEAR(buys.date) AS buy_year, MONTH(buys.date) AS buy_month,
       sells.price AS sell_price, buys.price AS buy_price, (sells.price - buys.price) AS diff, 
       qty, (sells.price * qty) AS sell_amt, (buys.price * qty) AS buy_amt, 
       (sells.price - buys.price) * qty AS gross, 
       ROUND((sells.price - buys.price)/buys.price*100,2) AS pct, profit AS net, buys.kind
FROM sells JOIN buys ON sells.buy_id = buys.id
JOIN stocks ON buys.stock_id = stocks.id
WHERE YEAR(sells.date) = %s
ORDER BY sells.date DESC, name'''
sql = sql % year
sells_df = pd.read_sql(sql, conpf)
print(sql)


SELECT name, YEAR(sells.date) AS sell_year, MONTH(sells.date) AS sell_month, 
       YEAR(buys.date) AS buy_year, MONTH(buys.date) AS buy_month,
       sells.price AS sell_price, buys.price AS buy_price, (sells.price - buys.price) AS diff, 
       qty, (sells.price * qty) AS sell_amt, (buys.price * qty) AS buy_amt, 
       (sells.price - buys.price) * qty AS gross, 
       ROUND((sells.price - buys.price)/buys.price*100,2) AS pct, profit AS net, buys.kind
FROM sells JOIN buys ON sells.buy_id = buys.id
JOIN stocks ON buys.stock_id = stocks.id
WHERE YEAR(sells.date) = 2022
ORDER BY sells.date DESC, name


In [3]:
sells_df.style.format(format_dict)

,name,sell_year,sell_month,buy_year,buy_month,sell_price,buy_price,diff,qty,sell_amt,buy_amt,gross,pct,net,kind
0,IMH,2022,1,2021,12,16.80,11.00,5.80,"1,500","25,200.00","16,500.00","8,700.00",52.73,"8,607.64",DTD
1,IMH,2022,1,2021,12,16.80,11.00,5.80,"1,500","25,200.00","16,500.00","8,700.00",52.73,"8,607.64",DTD
2,DCC,2022,1,2021,9,3.00,2.90,0.10,"40,000","120,000.00","116,000.00","4,000.00",3.45,"3,477.28",DTD
3,EPG,2022,1,2021,9,12.30,11.20,1.10,"2,000","24,600.00","22,400.00","2,200.00",9.82,"2,095.90",DTD
4,IVL,2022,1,2021,3,44.50,42.00,2.50,"3,000","133,500.00","126,000.00","7,500.00",5.95,"6,925.23",DTD


In [4]:
sells_df.groupby(['sell_year','sell_month','buy_year','buy_month','name']).net.sum()

sell_year  sell_month  buy_year  buy_month  name
2022       1           2021      3          IVL     6,925.23
                                 9          DCC     3,477.28
                                            EPG     2,095.90
                                 12         IMH    17,215.28
Name: net, dtype: float64

In [5]:
file_name = 'ytd-profit.csv'
data_file = data_path + file_name
csv_file = csv_path + file_name
box_file = box_path + file_name

sells_df.groupby(['sell_year','sell_month','buy_year','buy_month','name']).net.sum().to_csv(csv_file)
sells_df.groupby(['sell_year','sell_month','buy_year','buy_month','name']).net.sum().to_csv(data_file)
sells_df.groupby(['sell_year','sell_month','buy_year','buy_month','name']).net.sum().to_csv(box_file)

In [6]:
sells_df.groupby(['sell_month']).net.sum()

sell_month
1   29,713.69
Name: net, dtype: float64

In [7]:
profit = sells_df.net.sum()
ttl_profit = profit + dividend
profit,dividend,ttl_profit

(29713.69, 0.0, 29713.69)

In [8]:
profit = sells_df.net.sum()
net = profit + dividend - expense
profit,dividend,expense, net

(29713.69, 0.0, 20000.0, 9713.689999999999)

### Bottom line of up to date transactions

In [9]:
total = refund + net
total

9713.689999999999